In [1]:
%reload_ext autoreload
%autoreload all

%aimport -torch
%aimport -matplotlib
%aimport -seaborn
%aimport -numpy
%aimport -pandas
%aimport -scipy
%aimport -lightning 

In [2]:
from eeg_snn_encoder.config import PROCESSED_DATA_DIR
from eeg_snn_encoder.dataset import CHBMITDataModule, CHBMITDataset

# Load the dataset
dataset = CHBMITDataset(PROCESSED_DATA_DIR / "stft_normalized.h5")

2025-05-08 22:58:34.859 | INFO     | eeg_snn_encoder.config:<module>:11 - PROJ_ROOT path is: /home/jupyter-group55/snn-encoder-test


In [3]:
datamodule = CHBMITDataModule(dataset, batch_size=512, worker=0)

In [4]:
tbr_studies = [
    {"name": "model-tuning-tbr", "score_limit": lambda x: x.value < 15},
    {"name": "model-tuning-tbr-wrong", "score_limit": lambda x: x.value < 15},
    {"name": "model-tuning-tbr-mse", "score_limit": lambda x: x.value < 0.4},
]

In [5]:
import os

import optuna

test_trials = []

for i in tbr_studies:
    print(f"Loading good trial from {i['name']}")
    old_study = optuna.load_study(
        study_name=i["name"],
        storage=os.environ["OPTUNA_CONN_STRING"],
    )

    complete_trial = old_study.get_trials(False, states=(optuna.trial.TrialState.COMPLETE,))
    filtered_trials = list(filter(i["score_limit"], complete_trial))
    sorted_trials = sorted(filtered_trials, key=lambda t: t.value)

    test_trials += sorted_trials
    print(len(sorted_trials))

Loading good trial from model-tuning-tbr
5
Loading good trial from model-tuning-tbr-wrong
4
Loading good trial from model-tuning-tbr-mse
1


In [6]:
study = optuna.create_study(
    direction="minimize",
    study_name="model-fine-tuning-tbr",
    storage=os.environ["OPTUNA_CONN_STRING"],
    load_if_exists=True,
    pruner=optuna.pruners.NopPruner(),
)

[I 2025-05-08 22:58:51,795] A new study created in RDB with name: model-fine-tuning-tbr


In [7]:
import lightning.pytorch as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from loguru import logger
from optuna.integration import PyTorchLightningPruningCallback
import torch

from eeg_snn_encoder.encoders import TBREncoder
from eeg_snn_encoder.models.classifier import ModelConfig
from eeg_snn_encoder.models.lightning import LitSeizureClassifier, OptimizerConfig

torch.set_float32_matmul_precision("medium")


def objective(trial: optuna.Trial) -> float:
    model_params: ModelConfig = {
        "threshold": trial.suggest_float("threshold", 0.01, 0.5),
        "slope": trial.suggest_float("slope", 1.0, 20.0),
        "beta": trial.suggest_float("beta", 0.1, 0.99),
        "dropout_rate1": trial.suggest_float("dropout_rate1", 0.1, 0.99),
        "dropout_rate2": trial.suggest_float("dropout_rate2", 0.1, 0.99),
    }

    optimizer_params: OptimizerConfig = {
        "lr": trial.suggest_float("lr", 1e-6, 1e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-4, log=True),
        "scheduler_factor": trial.suggest_float("scheduler_factor", 0.1, 0.99),
        "scheduler_patience": trial.suggest_int("scheduler_patience", 1, 10),
    }

    encoder_params = {
        "threshold": trial.suggest_float("tbr_threshold", 0.1, 4),
    }

    spike_encoder = TBREncoder(**encoder_params)

    lit_model = LitSeizureClassifier(
        model_config=model_params,
        optimizer_config=optimizer_params,
        spike_encoder=spike_encoder,
    )

    trainer = pl.Trainer(
        max_epochs=50,
        accelerator="auto",
        devices="auto",
        strategy="auto",
        enable_model_summary=False,
        callbacks=[
            PyTorchLightningPruningCallback(trial, monitor="val_mse"),
            EarlyStopping(monitor="val_loss", mode="min", patience=10),
        ],
        logger=False,
    )

    trainer.fit(lit_model, datamodule=datamodule)

    return trainer.callback_metrics["val_mse"]

In [8]:
for trial in test_trials:
    study.enqueue_trial(params=trial.params)

study.optimize(objective, n_trials=len(test_trials))

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/jupyter-group55/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/jupyter-group55/snn-encoder-test/notebooks/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/jupyter-group55/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/home/jupyter-group55/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
[I 2025-05-08 23:14:37,117] Trial 9 finished with value: 0.2642762362957001 and parameters: {'threshold': 0.03080555005380059, 'slope': 14.518270811980521, 'beta': 0.7447525051066537, 'dropout_rate1': 0.6531982105581555, 'dropout_rate2': 0.6919872326298507, 'lr': 5.368802703748187e-05, 'weight_decay': 4.407533848519087e-06, 'scheduler_factor': 0.6149999778901825, 'scheduler_patience': 7, 'tbr_threshold': 1.9556269198948832}. Best is trial 4 with value: 0.22310757637023926.


In [9]:
logger.info(f"Encoder: TBR Encoding,trial, best_param: {study.best_params}")
logger.info(f"Encoder: TBR Encoding,trial, best_score: {study.best_value}")

2025-05-08 23:14:37.576 | INFO     | __main__:<module>:1 - Encoder: TBR Encoding,trial, best_param: {'threshold': 0.03291337141821968, 'slope': 2.092119829886898, 'beta': 0.9223367826247865, 'dropout_rate1': 0.46240133436195446, 'dropout_rate2': 0.10587513257010617, 'lr': 5.7554283743801334e-05, 'weight_decay': 2.5217067303532902e-06, 'scheduler_factor': 0.22054744646011884, 'scheduler_patience': 10, 'tbr_threshold': 0.16324340424833683}
2025-05-08 23:14:38.029 | INFO     | __main__:<module>:2 - Encoder: TBR Encoding,trial, best_score: 0.22310757637023926
